# **To what extent and accuracy can we predict the growth of a player based on the team and league they are a part of?**

Mount google drive.

In [83]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab\ Notebooks/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


Import Cleaned Data.

Since the statistics between goalies and other positions are so different, we are splitting our training data into goalie and other player positions. 

In [0]:
import pandas as pd 

goalie = pd.read_csv('refined_goalies.csv')
player = pd.read_csv('refined_players.csv')

Glance at data format.

In [85]:
goalie.head()

,Unnamed: 0,sofifa_id,club,overall,age,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,predicted_growth,year,actual_growth
0,0,167495,FC Bayern München,3.202943,0.335843,2.618522,2.711699,3.758156,2.221370,1.891575,3.004741,-0.905412,15,0
1,1,192119,Chelsea,2.679073,-0.814604,2.234057,2.965569,0.861386,2.462583,0.796741,2.541218,-0.121640,15,0
2,2,167948,Tottenham Hotspur,2.548106,0.144102,2.490367,2.203959,1.113279,2.462583,2.289696,1.961815,-0.709469,15,0
3,3,48940,Chelsea,2.548106,1.102808,2.105902,2.584764,1.365172,1.980158,0.796741,2.193576,-0.905412,15,0
4,4,168435,Paris Saint-Germain,2.417138,0.144102,2.362212,1.950089,1.365172,2.341977,0.995802,2.309457,-0.709469,15,-2


In [86]:
player.head()

,Unnamed: 0,sofifa_id,club,overall,age,player_positions,skill_moves,pace,shooting,passing,dribbling,defending,physic,predicted_growth,year,actual_growth
0,0,158023,FC Barcelona,3.783296,0.486906,CF,2.379525,2.187743,2.566249,2.662503,3.151238,-1.469820,-0.283486,-0.579009,15,1
1,1,20801,Real Madrid,3.637927,0.948639,"LW, LM",3.975020,2.187743,2.850949,2.189643,2.675592,-1.174914,1.386015,-0.975286,15,1
2,2,9014,FC Bayern München,3.347187,1.179505,"RM, LM, RW",2.379525,2.187743,2.352724,2.378787,2.770721,-1.174914,-0.179142,-0.975286,15,0
3,3,41236,Paris Saint-Germain,3.347187,1.641238,ST,2.379525,0.654661,2.708599,2.189643,2.199947,-1.056951,2.116421,-0.975286,15,-1
4,4,176580,FC Barcelona,3.201817,0.486906,"ST, CF",2.379525,1.285930,2.423899,2.000500,2.390205,-0.585100,1.386015,-0.579009,15,1


In [87]:
print(goalie.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8581 entries, 0 to 8580
Data columns (total 14 columns):
Unnamed: 0          8581 non-null int64
sofifa_id           8581 non-null int64
club                8581 non-null object
overall             8581 non-null float64
age                 8581 non-null float64
gk_diving           8581 non-null float64
gk_handling         8581 non-null float64
gk_kicking          8581 non-null float64
gk_reflexes         8581 non-null float64
gk_speed            8581 non-null float64
gk_positioning      8581 non-null float64
predicted_growth    8581 non-null float64
year                8581 non-null int64
actual_growth       8581 non-null int64
dtypes: float64(9), int64(4), object(1)
memory usage: 938.7+ KB
None


In [88]:
print(player.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69096 entries, 0 to 69095
Data columns (total 16 columns):
Unnamed: 0          69096 non-null int64
sofifa_id           69096 non-null int64
club                69096 non-null object
overall             69096 non-null float64
age                 69096 non-null float64
player_positions    69096 non-null object
skill_moves         69096 non-null float64
pace                69096 non-null float64
shooting            69096 non-null float64
passing             69096 non-null float64
dribbling           69096 non-null float64
defending           69096 non-null float64
physic              69096 non-null float64
predicted_growth    69096 non-null float64
year                69096 non-null int64
actual_growth       69096 non-null int64
dtypes: float64(10), int64(4), object(2)
memory usage: 8.4+ MB
None


Convert all object data columns to categorical/one-hot data dummy columns and drop sofifa ID because the specific player should not matter in this regression problem. Also get rid of the row number identifier because the dataframe comes with a built in row identifier and the row number should not be necessary in the regression problem.

In [0]:
from pandas import get_dummies

def all_categorical(data):
    object_columns = []
    for column in data:
        if data[column].dtype == object:
            object_columns.append(column)
    new_data = get_dummies(data, columns=object_columns)
    return new_data
  
player = all_categorical(player)
player = player.drop(['sofifa_id', 'Unnamed: 0'], axis=1)
goalie = all_categorical(goalie)
goalie = goalie.drop(['sofifa_id', 'Unnamed: 0'], axis=1)

Find which numerical features actually matter in the regression problem with the pearson correlation.

In [0]:
import seaborn as sns
%matplotlib inline

proxy_goalie = goalie[['actual_growth', 'predicted_growth', 'overall', 'age',	'gk_diving',	'gk_handling',	'gk_kicking',	'gk_reflexes',	'gk_speed', 'gk_positioning',	'year']].copy()
proxy_player = player[['actual_growth', 'predicted_growth', 'overall',	'age',	'skill_moves',	'pace',	'shooting', 'passing', 'dribbling', 'defending',	'physic',	'year',	]].copy()

cor_matrix_goalie = proxy_goalie.corr()
cor_matrix_player = proxy_player.corr()

In [91]:
cor_matrix_goalie

,actual_growth,predicted_growth,overall,age,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,year
actual_growth,1.000000,0.223470,-0.263216,-0.262318,-0.216453,-0.250132,-0.192233,-0.215507,-0.012190,-0.263653,-0.227743
predicted_growth,0.223470,1.000000,-0.534030,-0.855534,-0.461805,-0.477288,-0.414600,-0.461023,-0.264693,-0.558632,0.104562
overall,-0.263216,-0.534030,1.000000,0.484595,0.931615,0.915082,0.761280,0.930723,0.361435,0.931779,-0.007324
age,-0.262318,-0.855534,0.484595,1.000000,0.399777,0.447785,0.368675,0.383859,0.173019,0.531981,-0.001016
gk_diving,-0.216453,-0.461805,0.931615,0.399777,1.000000,0.794179,0.683776,0.899793,0.380668,0.810327,-0.064696
gk_handling,-0.250132,-0.477288,0.915082,0.447785,0.794179,1.000000,0.704322,0.786131,0.272803,0.857754,0.009729
gk_kicking,-0.192233,-0.414600,0.761280,0.368675,0.683776,0.704322,1.000000,0.674148,0.278961,0.698685,0.020653
gk_reflexes,-0.215507,-0.461023,0.930723,0.383859,0.899793,0.786131,0.674148,1.000000,0.371669,0.814086,-0.078306
gk_speed,-0.012190,-0.264693,0.361435,0.173019,0.380668,0.272803,0.278961,0.371669,1.000000,0.305744,-0.241635
gk_positioning,-0.263653,-0.558632,0.931779,0.531981,0.810327,0.857754,0.698685,0.814086,0.305744,1.000000,-0.014438


In [92]:
cor_matrix_player

,actual_growth,predicted_growth,overall,age,skill_moves,pace,shooting,passing,dribbling,defending,physic,year
actual_growth,1.000000,0.284499,-0.335919,-0.307164,-0.095021,-0.011429,-0.158719,-0.209740,-0.167487,-0.098171,-0.188072,-0.283976
predicted_growth,0.284499,1.000000,-0.500544,-0.864198,-0.107576,0.062920,-0.245525,-0.340113,-0.195452,-0.237225,-0.455937,0.051897
overall,-0.335919,-0.500544,1.000000,0.446806,0.370280,0.181318,0.476998,0.661018,0.577293,0.297993,0.489627,0.030128
age,-0.307164,-0.864198,0.446806,1.000000,0.069819,-0.192684,0.213582,0.308637,0.132512,0.249613,0.418507,0.045892
skill_moves,-0.095021,-0.107576,0.370280,0.069819,1.000000,0.405660,0.582253,0.505854,0.642463,-0.352154,-0.180863,0.060408
pace,-0.011429,0.062920,0.181318,-0.192684,0.405660,1.000000,0.365150,0.297148,0.566105,-0.309671,-0.190326,-0.049459
shooting,-0.158719,-0.245525,0.476998,0.213582,0.582253,0.365150,1.000000,0.644237,0.766261,-0.448397,-0.022963,-0.013512
passing,-0.209740,-0.340113,0.661018,0.308637,0.505854,0.297148,0.644237,1.000000,0.815497,0.102180,0.106676,-0.005330
dribbling,-0.167487,-0.195452,0.577293,0.132512,0.642463,0.566105,0.766261,0.815497,1.000000,-0.233600,-0.082133,0.013763
defending,-0.098171,-0.237225,0.297993,0.249613,-0.352154,-0.309671,-0.448397,0.102180,-0.233600,1.000000,0.553547,0.004179


For goalie numerical data, gk speed has little correlation with actual growth so we get rid of that.
For player numerical data, skill moves, defending, and pace have little to do with predicted growth so we also get rid of that.

In [0]:
goalie = goalie.drop(['gk_speed'], axis=1)
player = player.drop(['skill_moves', 'defending', 'pace'], axis=1)

Remove actual growth and put into its one pandas dataframe.

In [0]:
goalie_growth = goalie['actual_growth']
player_growth = player['actual_growth']

goalie = goalie.drop(['actual_growth'], axis=1)
player = player.drop(['actual_growth'], axis=1)

Linear Regression Model (Ordinary Least Squares)

We test this model to see if there is somehow a linear relationship between our features and the data.

In [99]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(goalie, goalie_growth, test_size=0.2, random_state=10)

Linear_Regression_Goalie = LinearRegression()
Linear_Regression_Goalie.fit(x_train, y_train)
y_predict = Linear_Regression_Goalie.predict(x_test)
mse = mean_squared_error(y_test, y_predict)
print(mse)

x_train, x_test, y_train, y_test = train_test_split(player, player_growth, test_size=0.2, random_state=10)

Linear_Regression_Player = LinearRegression()
Linear_Regression_Player.fit(x_train, y_train)
y_predict = Linear_Regression_Player.predict(x_test)
mse = mean_squared_error(y_test, y_predict)
print(mse)

7.392574599749646e+22
4.9696987914286855e+22


Clearly, there is not a linear relationship; therefore, we decided to use a model that is more robust in dealing with non-linear data.

Random Forest Regression Model. Regression model that fits non-linear data fairly well. 

In [100]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

x_train, x_test, y_train, y_test = train_test_split(goalie, goalie_growth, test_size=0.2, random_state=10)

forest_goalie = RandomForestRegressor(n_estimators = 100)
forest_goalie.fit(x_train, y_train)
y_predict = forest_goalie.predict(x_test)
mse = mean_squared_error(y_test, y_predict)
print(mse)

x_train, x_test, y_train, y_test = train_test_split(player, player_growth, test_size=0.2, random_state=10)

forest_player = RandomForestRegressor(n_estimators = 100)
forest_player.fit(x_train, y_train)
y_predict = forest_player.predict(x_test)
mse = mean_squared_error(y_test, y_predict)
print(mse)

4.775639021549213
4.22387510853835


Support Vector Regression. Regression model that fits well with high dimensional data (we have over 800 features due to one hot encoding all the categorical data) and is also robust at dealing with non-linear relationships.

In [0]:
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

x_train, x_test, y_train, y_test = train_test_split(goalie, goalie_growth, test_size=0.2, random_state=10)

svr_goalie = SVR(gamma='scale')
svr_goalie.fit(x_train, y_train)
y_predict = svr_goalie.predict(x_test)
mse = mean_squared_error(y_test, y_predict)
print(mse)

x_train, x_test, y_train, y_test = train_test_split(player, player_growth, test_size=0.2, random_state=10)

svr_player = SVR(gamma='scale')
svr_player.fit(x_train, y_train)
y_predict = svr_player.predict(x_test)
mse = mean_squared_error(y_test, y_predict)
print(mse)

4.756689449786852


Random Forest Regressor With Pruning to see if we overfitted our features.

In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

x_train, x_test, y_train, y_test = train_test_split(goalie, goalie_growth, test_size=0.2, random_state=10)

forest_goalie = RandomForestRegressor(n_estimators = 100, max_depth = 50)
forest_goalie.fit(x_train, y_train)
y_predict = forest_goalie.predict(x_test)
mse = mean_squared_error(y_test, y_predict)
print(mse)

x_train, x_test, y_train, y_test = train_test_split(player, player_growth, test_size=0.2, random_state=10)

forest_player = RandomForestRegressor(n_estimators = 100, max_depth = 50)
forest_player.fit(x_train, y_train)
y_predict = forest_player.predict(x_test)
mse = mean_squared_error(y_test, y_predict)
print(mse)